In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 18 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

20 February 2023 ; 19:29:58


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,26,0,9.4,1.6,3.5,0.457,...,3.22,114.51,111.37,3.14,3.11,114.52,111.48,3.05,1,1
9,MarJon Beauchamp,SF,22,MIL,38,9,14.8,2.1,5.3,0.398,...,3.22,114.51,111.37,3.14,3.11,114.52,111.48,3.05,1,1
15,A.J. Lawson,SG,22,DAL,13,0,4.8,1.0,2.0,0.500,...,0.48,117.31,116.83,0.48,0.65,117.22,116.60,0.63,1,1
23,Jaden Hardy,SG,20,DAL,31,0,11.8,2.2,5.2,0.422,...,0.48,117.31,116.83,0.48,0.65,117.22,116.60,0.63,1,1
34,AJ Griffin,SF,19,ATL,55,11,20.4,3.7,7.7,0.477,...,-0.36,115.47,115.82,-0.34,-0.58,115.53,116.11,-0.57,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.7,ORL,15.0,0.315385,2.680769,6.5,19.9,13,0.243478
1,Bennedict Mathurin,28.2,IND,13.6,-0.683333,2.323333,5.5,17.2,12,0.139392
2,Walker Kessler,21.3,UTA,21.0,1.870175,7.049123,3.6,8.2,11,0.123886
3,Jalen Duren,25.2,DET,16.8,0.643137,5.627451,3.6,8.6,15,0.072555
4,Jaden Ivey,30.1,DET,12.2,-0.745455,0.298182,5.2,15.2,15,0.063862


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)